In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from IPython.display import Markdown

%matplotlib inline
%config InlineBackend.figure_format='svg'

from genominterv.decorators import bootstrap
from genominterv.stats import proximity_stat, jaccard_stat

from scipy.stats import fisher_exact

import geneinfo.plot as gplt
import geneinfo.information as gi
import geneinfo.utils as utils
# utils.clear_cache()
# utils.disable_cache = True
from geneinfo.utils import GeneListCollection
from geneinfo.utils import GeneList as glist

sheet = GeneListCollection(google_sheet='1JSjSLuto3jqdEnnG7JqzeC_1pUZw76n7XueVAYrUOpk')

def fisher(background_set, set_a, set_b):
    M = len(background_set) 
    N = len(background_set.intersection(set_a)) 
    n = len(background_set.intersection(set_b))
    x = len(background_set.intersection(set_b).intersection(set_a))
    table = [[  x,           n - x          ],
             [ N - x,        M - (n + N) + x]]
    return table, fisher_exact(table, alternative='greater').pvalue, glist(sorted(set_a.intersection(set_b)))

# Gene lists

In [ ]:
#| label: tbl-ech90-genelist
sheet.get('ech90_regions') << sheet.get('cDEG') << sheet.get('xi_escape') << sheet.get('gametologs')

# Human-Neanderthal introgression

Also include these in the admixture project and import to this:

```txt
argweaver-d/introgressionHub/files/M1A/afr1.bed
argweaver-d/introgressionHub/files/M1A/afr2.bed
argweaver-d/introgressionHub/files/M1A/any.bed
argweaver-d/introgressionHub/files/M1A/den.bed
argweaver-d/introgressionHub/files/M1A/notfixed.bed
```

In [ ]:
altai_segments = pd.read_csv('../data/argweaver-d/introgressionHub/files/M1A/alt.bed', 
                             sep='\t', names=['chrom', 'start', 'end', 'label', 'X1', 'X2', 'X3', 'X4', 'X5'])
vindia_segments = pd.read_csv('../data/argweaver-d/introgressionHub/files/M1A/vin.bed', 
                             sep='\t', names=['chrom', 'start', 'end', 'label', 'X1', 'X2', 'X3', 'X4', 'X5'])

In [ ]:
altai_het_coord = altai_segments.loc[altai_segments.label.str.startswith('humToAltNea.het'), ['chrom', 'start', 'end', 'label']]
altai_hom_coord = altai_segments.loc[altai_segments.label.str.startswith('humToAltNea.hom'), ['chrom', 'start', 'end', 'label']]
vindia_het_coord = vindia_segments.loc[vindia_segments.label.str.startswith('humToVinNea.het'), ['chrom', 'start', 'end', 'label']]
vindia_hom_coord = vindia_segments.loc[vindia_segments.label.str.startswith('humToVinNea.hom'), ['chrom', 'start', 'end', 'label']]

In [ ]:
plot_df = pd.concat([altai_segments, vindia_segments])
plot_df['kind'] = [x.rsplit('.', 1)[0] for x in plot_df.label]
plot_df['length'] = plot_df.end - plot_df.start
plot_df['chrom_group'] = plot_df.chrom
plot_df.loc[plot_df.chrom.isin(list(f'chr{x}' for x in range(1,23))), 'chrom_group'] = 'autosome'

In [ ]:
#| label: fig-hum-nean-length-distribution

g = sns.FacetGrid(data=plot_df.loc[plot_df.label.str.startswith('hum')], col='kind', 
                  col_wrap=2, hue='chrom_group', aspect=2)
g.map(sns.histplot, 'length', stat='density', bins=range(0, 700000, 50000)) ;

In [ ]:
altai_annot = [(x.chrom, x.start, x.end) for x in altai_hom_coord.itertuples()] + \
    [(x.chrom, x.start, x.end) for x in altai_het_coord.itertuples()]
vindia_annot = [(x.chrom, x.start, x.end) for x in vindia_hom_coord.itertuples()] + \
    [(x.chrom, x.start, x.end) for x in vindia_het_coord.itertuples()]

#gi.chrom_ideogram(altai_annot + vindia_annot, figsize=(12,8))

In [ ]:
#| label: fig-hum-nean-genome-ideogram-tall
#| fig-cap: "Modern human introgression into Neanderthals ~250,000 BP"

g = gplt.GenomeIdeogram(assembly='hg19', axes_height_inches=0.4, axes_width_inches=8)
g.draw_chromosomes(height=4)
g.add_segments(altai_annot, facecolor='red', base=4, height=2, label='Altai')
g.add_segments(vindia_annot, facecolor='blue', base=6, height=2, label='Vindia')
#gid.add_legend()

In [ ]:
#| label: fig-hum-nean-genome-ideogram

g = gplt.GenomeIdeogram(assembly='hg19', axes_height_inches=0.3) 
g.draw_chromosomes(height=4)
g.add_segments(altai_annot, facecolor='red', base=4, height=2, label='Altai')
g.add_segments(vindia_annot, facecolor='blue', base=6, height=2, label='Vindia')
#gid.add_legend()

In [ ]:
g = gplt.ChromIdeogram('chrX', assembly='hg19') 
g.draw_chromosomes()
g.add_segments([x for x in altai_annot if x[0] == 'chrX'], facecolor='red', base=4, height=1, label='Altai')
g.add_segments([x for x in vindia_annot if x[0] == 'chrX'], facecolor='blue', base=5, height=1, label='Vindia')
g.add_legend()

In [ ]:
import geneinfo.information as gi
import geneinfo.plot as gplt

In [ ]:
g = gplt.ChromIdeogram('chr6', assembly='hg19',
                      zooms=[(80_000_000, 90_000_000)]) 
g.draw_chromosomes()
g.add_segments([x for x in altai_annot if x[0] == 'chr6'], facecolor='red', base=4, height=1, label='Altai')
g.add_segments([x for x in vindia_annot if x[0] == 'chr6'], facecolor='blue', base=5, height=1, label='Vindia')
g.add_labels(gi.gene_labels_region('chr6', 80_000_000, 90_000_000, assembly='hg19'))
g.add_legend()

In [ ]:
lst = []
for chrom, start, end in altai_annot:
    if chrom != 'chrX':
        continue
    lst.extend([x[0] for x in gi.gene_coords_region(chrom, start, end, assembly='hg19')])
hum_altai_genes = glist(sorted(set(lst)))
hum_altai_genes

In [ ]:
lst = []
for chrom, start, end in vindia_annot:
    if chrom != 'chrX':
        continue
    lst.extend([x[0] for x in gi.gene_coords_region(chrom, start, end, assembly='hg19')])
hum_vindia_genes = glist(sorted(set(lst)))
hum_vindia_genes

In [ ]:
hum_nean_genes = hum_altai_genes | hum_vindia_genes
hum_nean_genes

**NB:** removing microRNA genes from set overlapping human to neanderthal admixture (for a fair comparison since Meritxell is not able to find miRNAs):

In [ ]:
human_nean_genes = glist([g for g in hum_nean_genes if not g.startswith('MIR')])
human_nean_genes << hum_altai_genes

## Overlap hum-nean with cDEG genes

In [ ]:
#| label: fig-hum-nean-cDEG-overlap

g = gplt.ChromIdeogram('chrX', assembly='hg19', #rel_font_height=0.04,
    zooms=[
        (19_000_000, 22_000_000),
        (37_000_000, 39_000_000),
        (48_000_000, 50_000_000),
        (96_000_000, 98_000_000),
        (107_000_000, 108_000_000),
        (114_000_000, 115_000_000),
        (118_000_000, 120_000_000),
        (128_000_000, 130_000_000),
    ]
)
g.draw_chromosomes()
g.add_segments(list(filter(lambda x: x[0] == 'chrX', altai_annot)), facecolor='tab:orange', base=4, height=1, label='Altai', alpha=0.7)
g.add_segments(list(filter(lambda x: x[0] == 'chrX', vindia_annot)), facecolor='tab:green', base=5, height=1, label='Vindia', alpha=0.7)

# def color(name):
#     return 'black' if name in hum_nean_genes else 'black'

coords = gi.gene_labels(sheet.get('cDEG'), assembly='hg19')
g.add_labels(coords, base=g.ideogram_base, min_height=g.ideogram_height*1.5, zorder=10, color='black')
# coords.extend(
#     [(tup[0], (tup[1]+tup[2])/2, name) for name, tup in gi.gene_coord(cDEG_genes, species='homo_sapiens', assembly='hg19').items()]
# )
# g.add_labels(coords, base=g.ideogram_base, min_height=g.ideogram_height*1.5, zorder=10, color='red')
g.add_legend()

In [ ]:
#| label: doc-hum-nean-cDEG-overlap-fisher

human_nean_genes << sheet.get('cDEG')

In [ ]:
human_nean_genes.fisher(sheet.get('cDEG'), sheet.get('all_npx'))

## Overlap hum-nean with nDEG genes

In [ ]:
g = gplt.ChromIdeogram('chrX', assembly='hg19', #rel_font_height=0.04,
    zooms=[
        (11_000_000, 14_000_000),
        (19_000_000, 22_000_000),
        (37_000_000, 39_000_000),
        (48_000_000, 50_000_000),
        (96_000_000, 98_000_000),
        (107_000_000, 109_000_000),
        (114_000_000, 116_000_000),
        (118_000_000, 121_000_000),
        (128_000_000, 131_000_000),
        (135_000_000, 137_000_000),
        (144_000_000, 152_000_000),
    ]
)
g.draw_chromosomes()
g.add_segments(list(filter(lambda x: x[0] == 'chrX', altai_annot)), facecolor='tab:orange', base=4, height=1, label='Altai', alpha=0.7)
g.add_segments(list(filter(lambda x: x[0] == 'chrX', vindia_annot)), facecolor='tab:green', base=5, height=1, label='Vindia', alpha=0.7)

coords = gi.gene_labels(sheet.get('nDEG'), assembly='hg19')
g.add_labels(coords, base=g.ideogram_base, min_height=g.ideogram_height*1.5, zorder=10, color='black')
g.add_legend()

In [ ]:
human_nean_genes.fisher(sheet.get('nDEG'), sheet.get('all_npx'))

## cDEG eith human-neanm Xie, Gametologs, Xi_copy_modul

In [ ]:
sheet.get('xi_escape') << sheet.get('nDEG')

In [ ]:
sheet.get('xi_escape').fisher(sheet.get('nDEG'), sheet.get('all_npx'))

In [ ]:
sheet.get('gametologs') << sheet.get('nDEG')

In [ ]:
sheet.get('gametologs').fisher(sheet.get('nDEG'), sheet.get('all_npx'))

In [ ]:
sheet.get('expr_mod_xi_copynr')

In [ ]:
sheet.get('xi_escape') & sheet.get('gametologs') & sheet.get('expr_mod_xi_copynr')

In [ ]:
(sheet.get('xi_escape') & sheet.get('gametologs') & sheet.get('expr_mod_xi_copynr')).fisher(sheet.get('nDEG'), sheet.get('all_npx'))

In [ ]:
(sheet.get('xi_escape') & sheet.get('gametologs')).fisher(sheet.get('nDEG'), sheet.get('all_npx'))

In [ ]:
sheet.get('expr_mod_xi_copynr')

In [ ]:
sheet.get('expr_mod_xi_copynr').fisher(sheet.get('cDEG'), sheet.get('all_npx'))

In [ ]:
sheet.get('accel_reg_simiiformes_br')

In [ ]:
sheet.get('accel_reg_simiiformes_br').fisher(sheet.get('cDEG'), sheet.get('all_npx'))

In [ ]:
sheet.get('ari_nonPUR')

In [ ]:
sheet.get('ari_nonPUR').fisher(sheet.get('cDEG'), sheet.get('all_npx'))

In [ ]:
((sheet.get('ech90_regions') | hum_nean_genes) & sheet.get('xi_escape')).fisher(sheet.get('cDEG'), sheet.get('all_npx'))

In [ ]:
hum_nean_genes.fisher(sheet.get('cDEG'), sheet.get('all_npx'))

# Macaque A/B compartments

## Functions for parsing compartment data and for statistics

In [ ]:
def plot_intervals(query=None, annot=None, **kwargs):

    import matplotlib.pyplot as plt

    vlines = kwargs.get('vlines', [])
    if 'vlines' in kwargs: del kwargs['vlines']
    figsize = kwargs.get('figsize', (8, 1.5*len(kwargs)-1))
    if 'figsize' in kwargs: del kwargs['figsize']
    scale_col = kwargs.get('scale', 1)
    if 'scale' in kwargs: del kwargs['scale']

    tups = list(kwargs.items())
    tups = reversed(tups)

    df_list = []
    labels = []
    for label, df in tups:
        labels.append(label)
        df['label'] = np.repeat(label, df.index.size)
        df_list.append(df)
    bigdf = pd.concat(df_list)

    bigdf['chrom'] = pd.Categorical(bigdf['chrom'], bigdf['chrom'].unique())
    bigdf['label'] = pd.Categorical(bigdf['label'], bigdf['label'].unique())

    gr = bigdf.groupby('chrom', observed=False)

    fig, axes = plt.subplots(gr.ngroups, 1, figsize=figsize, 
                            sharey=True
                            #  sharex=True
                             )
    if type(axes) is not np.ndarray:
        # in case there is only one axis so it not returned as a list
        axes = [axes]
    
    for i, chrom in enumerate(gr.groups):
        _df = gr.get_group(chrom)
        _gr = _df.groupby('label', observed=False)
        for y, label in enumerate(_gr.groups):
            try:
                df = _gr.get_group(label)
            except KeyError:
                continue
            y = np.repeat(y, df.index.size)
            for j in range(len(df.start)):
                # axes[i].hlines(y, df.start.tolist(), df.end.tolist(), alpha=df[alpha_col], lw=10, colors=f'C{y[0]}', capstyle='butt')
                axes[i].hlines(y, df.start.iloc[j], df.end.iloc[j], lw=10*df[scale_col].iloc[j], colors=f'C{y[0]}', capstyle='butt')
            delta = len(labels)/10

        axes[i].spines['top'].set_visible(False)
        axes[i].spines['left'].set_visible(False)
        axes[i].spines['right'].set_visible(False)

        axes[i].set_yticks(list(range(len(labels))), labels)
        axes[i].tick_params(axis='y', which='both', left=False)
        axes[i].set_ylim(-1, len(labels)-0.7)
        # axes[i].set_xlim(df.start.min()-delta, df.end.max()+delta)
        if i != gr.ngroups-1:
            axes[i].tick_params(axis='x', which='both', bottom=False)

        axes[i].set_title(chrom, loc='left', fontsize=10)

    for y, ax in enumerate(axes):
        y = np.repeat(y, len(vlines))
        axes[i].vlines(vlines, *ax.get_ylim(), lw=0.1, colors='black', zorder=0)
    
    plt.tight_layout()
    return axes

def stairs(df, start='start', end='end', pos='pos', endtrim=0):
    "Turn a df with start, end into one with pos to plot as stairs"
    df1 = df.copy(deep=True)
    df2 = df.copy(deep=True)
    df1[pos] = df1[start]
    df2[pos] = df2[end] - endtrim
    return pd.concat([df1, df2]).sort_values([start, end])
    
def parse_compartment_data(file_name):
    e1_100kb = pd.read_csv(file_name)
    e1_100kb['start'] = [i*100_000 for i in range(e1_100kb.index.size)]
    e1_100kb['end'] = e1_100kb.start + 100_000
    e1_100kb['sign'] = np.sign(e1_100kb.e1)
    e1_100kb['segment_id'] = ((e1_100kb.sign.shift() != e1_100kb.sign)).cumsum()
    
    comp = e1_100kb.groupby('segment_id', as_index=False).agg(dict(
         e1=['mean', 'sum'], 
         start='min', 
         end='max', 
         segment_id='mean', 
         sign='mean'
    ))
    comp.columns = ['_'.join(col).strip() for col in comp.columns.values]
    comp = comp.rename(
        columns={'start_min':'start',
                 'end_max':'end', 
                 'segment_id_mean':'segment_id', 
                 'sign_mean':'sign'}
    )
    comp['comp'] = ['A' if x > 0 else 'B' for x in comp.sign]
    comp = comp.reset_index()
    comp['chrom'] = 'chrX'
    
    _comp = comp.copy()
    for i in range(1, _comp.index.size-1):
        if np.isnan(_comp.loc[i-1, 'e1_mean']):
            _comp.loc[i, 'start'] = np.nan
        if np.isnan(_comp.loc[i+1, 'e1_mean']):
            _comp.loc[i, 'end'] = np.nan
    _comp = _comp.loc[~_comp.e1_mean.isnull(), :]
    _comp = _comp.reset_index()
    compartment_edges = pd.concat([_comp.start, _comp.end]).sort_values().unique()
    
    compartments = comp.loc[~comp.e1_mean.isnull()].copy()
    compartments['start'] = compartments.start.astype(int)
    compartments['end'] = compartments.end.astype(int)

    return compartments, compartment_edges

def edge_segments(compartment_edges, flank):
    compartment_edge_segm = pd.DataFrame(np.column_stack((compartment_edges, compartment_edges+flank)), columns=['start', 'end'])
    compartment_edge_segm['chrom'] = 'chrX'
    return compartment_edge_segm

chrom_sizes = {
    'chr1': 223616942, 'chr2': 196197964, 'chr5': 187317192, 'chr3': 185288947,
    'chr6': 179085566, 'chr4': 169963040, 'chr7': 169868564, 'chrX': 153388924,
    'chr8': 145679320, 'chr9': 134124166, 'chr11': 133066086, 'chr12': 130043856,
    'chr14': 128056306, 'chr15': 113283604, 'chr13': 108737130, 'chr10': 99517758,
    'chr17': 95433459, 'chr16': 79627064, 'chr20': 77137495, 'chr18': 74474043,
    'chr19': 58315233, 'chrY': 11753682,
}

@bootstrap(chrom_sizes)
def proximity_test(q, a):
    return proximity_stat(q, a)


@bootstrap(chrom_sizes)
def jaccard_test(q, a):
    return jaccard_stat(q, a)


def overlaps(df1, df2):
    """
    Establishes whether each query segment overlaps at least one 
    annotation segment. Returns a boolean array with same length 
    as df1.index.
    """
    overlapping = []
    for i, (s1, e1) in enumerate(zip(df1.start, df1.end)):
        overlaps = False
        for s2, e2 in zip(df2.start, df2.end):
            if e1 > s2 and e2 > s1:
                overlaps = True
                break
        overlapping.append(overlaps)
    return np.array(overlapping)


def svedig_tabel(orig_df, index, columns, values, cmap='Reds'):
    df = (orig_df
     .assign(log10p=np.log10(all_tests.p))
     .loc[(all_tests.p < 0.05)]
     .pivot(index=index, columns=columns, values=values)
    )
    df = df.rename(columns = {x:x.replace('_', '<br>') for x in df.columns.tolist()})
    df = (df.style
     .background_gradient(subset=df.columns, axis=None, cmap=cmap, vmin=0)
     .map(lambda x: 'color: transparent; background-color: transparent' if np.isnan(x) else '')
     .format('{:.3f}')
     .set_table_styles(
                {c: [{'selector': '', 
                      'props': [('min-width', '100px')],
                     }] for c in df.columns}, overwrite=False
     )
    )
    return df


## Annotation mapped to rheMac10

In [ ]:
ech90_human_Mmul_10 = pd.read_csv('../data/ech90_human_Mmul_10.csv')

flank = 1
high_hama_rhemac10 = pd.read_csv('../data/lift/rheMac10/high_hama_rhemac10.bed', sep='\t', 
            header=None, names=['label', 'chrom', 'start', 'end'])

hama_edges = np.concatenate((high_hama_rhemac10.start, high_hama_rhemac10.end))
hama_edge_1bp = pd.DataFrame(np.column_stack((hama_edges, hama_edges+flank)), columns=['start', 'end'])
hama_edge_1bp.start = np.maximum(hama_edge_1bp.start, 0)
hama_edge_1bp.end = np.minimum(hama_edge_1bp.end, chrom_sizes['chrX']) # rheMac10 chrX length
hama_edge_1bp['chrom'] = 'chrX'
hama_edge_1bp = hama_edge_1bp.sort_values(by=['start', 'end'])

high_olive_rhemac10 = pd.read_csv('../data/lift/rheMac10/high_olive_rhemac10.bed', sep='\t', 
            header=None, names=['label', 'chrom', 'start', 'end'])
olive_edges = np.concatenate((high_olive_rhemac10.start, high_olive_rhemac10.end))
olive_edge_1bp = pd.DataFrame(np.column_stack((olive_edges, olive_edges+flank)), columns=['start', 'end'])
olive_edge_1bp.start = np.maximum(olive_edge_1bp.start, 0)
olive_edge_1bp.end = np.minimum(olive_edge_1bp.end, chrom_sizes['chrX']) # rheMac10 chrX length
olive_edge_1bp['chrom'] = 'chrX'
olive_edge_1bp = olive_edge_1bp.sort_values(by=['start', 'end'])

## Tests

In [ ]:
if False:
    records = []
    for tissue in ['fibroblast', 'pachytene_spermatocyte', 'round_spermatid', 'sperm', 'spermatogonia']:
        for pc in ['arms', '10Mb']:
            file_name = f"../data/rec_compartments/{tissue}_e1_100kb_{pc}.csv"
    
            # get compartment data
            compartments, compartment_edges = parse_compartment_data(file_name)
            compartment_edge_segments = edge_segments(compartment_edges, flank=200_000)
            compartment_edge_1bp = edge_segments(compartment_edges, flank=1)
    
            # ech overlap to compartment edges
            query = ech90_human_Mmul_10
            annot = compartment_edge_1bp
            stat, p = jaccard_test(query, annot)
            records.append((tissue, pc, 'ECH90', 'comp_edge_1bp', 'jaccard', stat, p))
    
            # hama edge proximity to compartment edges
            query = hama_edge_1bp
            annot = compartment_edge_1bp
            stat, p = proximity_test(query.loc[~overlaps(query, annot)], annot)
            records.append((tissue, pc, 'hama_edge_1bp', 'comp_edge_1bp', 'proximity', stat, p))
            
            # olive edge proximity to compartment edges
            query = olive_edge_1bp
            annot = compartment_edge_1bp
            stat, p = proximity_test(query.loc[~overlaps(query, annot)], annot)
            records.append((tissue, pc, 'olive_edge_1bp', 'comp_edge_1bp', 'proximity', stat, p))
    
            # olive+hama edge proximity to compartment edges
            query = pd.concat([olive_edge_1bp, olive_edge_1bp]).sort_values(['start', 'end'])
            annot = compartment_edge_1bp
            stat, p = proximity_test(query.loc[~overlaps(query, annot)], annot)
            records.append((tissue, pc, 'olivehama_edge_1bp', 'comp_edge_1bp', 'proximity', stat, p))
    
    all_tests = pd.DataFrame().from_records(records, 
                                columns=['tissue', 'pc_scale', 'query',
                                         'annot', 'test', 'value', 'p'])
    all_tests['-log10p'] = -np.log10(all_tests.p) 
    all_tests.to_csv('all_tests.csv')
else:
    all_tests = pd.read_csv('../results/all_tests.csv')

In [ ]:
(all_tests.style
     .background_gradient(subset='-log10p', cmap='Reds', vmin=0)
     .format('{:.3f}', subset='-log10p')
     .map(func=lambda x: 'color: transparent; background-color: transparent' if x < 0.05 else '', subset='-log10p')
)

## Significant p-values 

In [ ]:
all_tests['tissue'] = pd.Categorical(all_tests.tissue, 
                                     categories=['fibroblast', 'spermatogonia', 
                                     'pachytene_spermatocyte', 'round_spermatid', 'sperm'])

In [ ]:
svedig = svedig_tabel(all_tests, index=['pc_scale', 'query', 'test' ], columns=["tissue"], values="p", cmap='Reds_r')
svedig

In [ ]:
# from IPython.display import Markdown, display
# df = (all_tests
#  .assign(log10p=np.log10(all_tests.p))
#  .loc[(all_tests.p < 0.05)]
#  .pivot(index=['pc_scale', 'query', 'test' ], columns=["tissue"], values="p")
# ).reset_index().fillna('-')

# # df.columns.name = None
# # df = df.reset_index().fillna('')
# # df = df.rename(columns={'pc_scale': 'scale'})
# def embed_table(df):
#     display(Markdown(df.to_markdown(index=False)))
    
# df << nice()

In [ ]:
df = (all_tests
 .assign(log10p=np.log10(all_tests.p))
 .loc[(all_tests.p < 0.05)]
 .pivot(index=['pc_scale', 'query', 'test' ], columns=["tissue"], values="p")
)
df.columns.name = None
df = df.reset_index().fillna('')
df = df.rename(columns={'pc_scale': 'scale'})
df.style.hide()



#df.columns = [' '.join(col).strip() for col in df.columns.values]

# #.style.hide()
# import textwrap
# df = df.rename(columns = {x:x.replace('_', ' ') for x in df.columns.tolist()})

# df.columns = df.columns.str.wrap(12, break_long_words=False)
# df.columns = df.columns.str.replace('\n', '<br>')

#df.columns = [textwrap.wrap(x, 10, break_long_words=False) for x in df.columns]


In [ ]:
#| label: tbl-ABborder-tests

df.loc[df.scale == '10Mb'].style.hide()

# Pure segments in baboon cooridnates

In [ ]:
from pandas.api.types import is_object_dtype
from math import floor, log10

class style():

    def __init__(self, subset=None, cmap='Reds', vmin=None, vmax=None, axis=0, **kwargs):
        self.subset = subset
        self.cmap = cmap
        self.vmin = vmin
        self.vmax = vmax
        self.kwargs = kwargs
        self.axis = axis

    def __rlshift__(self, df):
        "Left align columns of params frame: df << nice()"

        def make_pretty(styler):

            def commas(v):
                if type(v) is int:
                    s = str(v)[::-1]
                    return ','.join([s[i:i+3] for i in range(0, len(s), 3)])[::-1]
                elif type(v) is float:
                    signif_digits = 3
                    s = signif_digits - int(floor(log10(abs(v))))
                    return round(v, s)
                else:
                    return v                    

            return styler.format(commas)


        def left_align(styler):
            if self.subset is None:
                subset = df.columns           
            elif type(self.subset) is str:
                subset = [str]
            else:
                subset = self.subset
            
            return styler.set_table_styles(
                {c: [{'selector': '', 'props': [('text-align', 'left')]}] 
                     for c in df.columns if is_object_dtype(df[c]) and c != 'strand'},
                overwrite=False
            )

        def highlight(styler):
            return (styler
                  .background_gradient(subset=self.subset, cmap=self.cmap, axis=self.axis, vmin=self.vmin, vmax=self.vmax)
                  .map(func=lambda x: 'color: transparent; background-color: transparent' if x < 0.05 else '', 
                       subset=self.subset)
                 )
            
        s = df.style
        s = s.pipe(make_pretty)
        s = s.pipe(highlight)
        s = s.pipe(left_align)

        s = s.set_table_styles(
                {c: [{'selector': '', 'props': [('font-variant-numeric', 'tabular-nums')]}] 
                     for c in df.columns},
                overwrite=False
        )

        display(s)
        

#value_matrix_classification.style.apply(highlight_cells)
        
test = all_tests.iloc[:5].copy()
test['number'] = [x * 1000000 for x in range(4)] + [1111111]
        
test << style(subset='-log10p', cmap='Blues')
# test << nice(cmap='Blues', axis=None)

#test << nice(subset='test', cmap='Blues')

In [ ]:
#| label: tbl-pure-ancestry-border-tests

test = all_tests.iloc[:5].copy()
test['number'] = [x * 1000000 for x in range(4)] + [1111111]    
test

In [ ]:

class embedable():

    def __init__(self, signif_digits=3, int_commas=True):

        self.signif_digits = signif_digits
        self.int_commas = int_commas              

    def __rlshift__(self, df):
        "Left align columns of params frame: df << nice()"

        def signif_dig(v):
            if not v:
                return v
            s = self.signif_digits - int(floor(log10(abs(v))))
            return round(v, s)

        def commas(v):
            s = str(v)[::-1]
            return ','.join([s[i:i+3] for i in range(0, len(s), 3)])[::-1]            

        self.df = df.copy()  
        colalign = ['left' if t == 'object' else 'right' for t in df.dtypes ]

        for i, (c, t) in enumerate(zip(df.columns, df.dtypes)):
            if t == 'float64' and self.signif_digits is not None:
                self.df.loc[:, c] = self.df.loc[:, c].apply(signif_dig)
            if t == 'int64' and self.int_commas:
                self.df.loc[:, c] = self.df.loc[:, c].apply(commas)

                width = max(self.df[c].values, key=len)
                colalign[i] = 'right'

        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(Markdown(self.df.to_markdown(index=False, colalign=colalign)))
        

df.head(10) << embedable()

## Read in baboon LAI

In [ ]:
df = pd.read_hdf('../data/mean_window_df_eth.h5')
meta_data_samples = pd.read_csv("../data/Papio_metadata_with_clustering.txt", sep =" ")
gog_olives = meta_data_samples.loc[meta_data_samples.Origin == "Gog Woreda, Gambella region, Ethiopia"].PGDP_ID
chrX_lai_eth = df.loc[df.individual.isin(gog_olives) & (df.chrom == 'all_chrX')].groupby(["chrom", "individual", "start", "end"]).mean().reset_index()
chrX_lai_eth.head()

In [ ]:
win_means = chrX_lai_eth.groupby(['start', 'end']).north_sum.mean().reset_index()
win_means.head()

In [ ]:
#| label: fig-gog-ancestry-prop

plot_df = stairs(win_means)
plt.figure(figsize=(12, 4))
plt.fill_between(plot_df.pos, plot_df.north_sum)
plt.axhline(y=95000)

In [ ]:
win_means.head()

In [ ]:
win_means['pure'] = (win_means.north_sum == 0) | (win_means.north_sum >= 95000)

win_means['segment_id'] = (win_means.pure.shift() != win_means.pure).cumsum()

pure = win_means.groupby('segment_id', as_index=False).agg(dict(
     start='min', 
     end='max', 
     north_sum='mean', 
     pure='mean'
))

pure = pure.loc[pure.pure == 1, :]
pure['species'] = ['hama' if x >= 95000 else 'olive' for x in pure.north_sum]
pure['chrom'] = 'chrX'
pure['start'] = pure.start.astype(int)
pure['end'] = pure.end.astype(int)
pure['north_sum'] = pure.north_sum.astype(int)
olive_pure_segments = pure.loc[pure.species == 'olive', ['chrom', 'start', 'end', 'north_sum']]
hama_pure_segments = pure.loc[pure.species == 'hama', ['chrom', 'start', 'end', 'north_sum']]

In [ ]:
hama_pure_segments.head(3)

In [ ]:
olive_pure_segments.head(3)

In [ ]:
#| label: fig-gog-pure-regions

edges = pd.concat(
    [olive_pure_segments.start, olive_pure_segments.end, hama_pure_segments.start, hama_pure_segments.end, 
    ]).sort_values().unique()

axes = plot_intervals(**{
    'Pure Olive':olive_pure_segments.assign(scale=np.repeat(1, len(olive_pure_segments.north_sum))),
    '95% Hamadryas':hama_pure_segments.assign(scale=(hama_pure_segments.north_sum-95000)/5000),
    'vlines':edges,
    'figsize':(12, 2),
    'scale': 'scale'
    }
)

In [ ]:
lst = []
for tup in hama_pure_segments.itertuples():
    lst.extend([x[0] for x in gi.gene_coords_region(tup.chrom, tup.start, tup.end, assembly='papAnu4')])
hama_genes = glist(sorted(set(lst)))
hama_genes

In [ ]:
# import requests, sys

# def lift(chrom, start, end, lift_from, lift_to):
#     server = "https://rest.ensembl.org"
#     ext = f"/map/human/{lift_from}/{chrom.replace('chr', '')}:{start}..{end}:1/{lift_to}?"
     
#     r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
     
#     if not r.ok:
#       r.raise_for_status()
#       sys.exit()
     
#     decoded = r.json()
#     return decoded

# lift('chrX', 1000000, 2000000, lift_from='GRCh37', lift_to='GRCh38')

In [ ]:
# import geneinfo.information as gi

# g = gplt.ChromIdeogram('chrX',
#                        # assembly='hg19')
#                     species='Macaca mulatta')
#                     # species='Homo sapiens')
# g.draw_chromosomes()

# coords = [(tup[0], (tup[1]+tup[2])/2, name) for name, tup in gi.gene_coord(cDEG_genes, species='macaca_mulatta').items()]
# g.add_labels(coords)

# coords = [(tup[0], (tup[1]+tup[2])/2, name) for name, tup in gi.gene_coord(hama_genes, species='macaca_mulatta').items()]
# g.add_labels(coords, color='tab:blue')


# g.add_segments([(x.chrom, x.start, x.end) for x in high_hama_rhemac10.itertuples()])
# #g.add_segments([(x.chrom, x.start, x.end) for x in hama_pure_segments.itertuples()], color='red', alpha=0.5)

In [ ]:
sojern_genes = glist([
    'AKAP4', 'ALG13', 'ATP7A', 'ATRX', 'BRCC3', 'CCNB3', 'CENPVL3', 'CLCN5', 'CLCN5', 
    'CMC4', 'COX7B', 'CYBB', 'DKC1', 'DYNLT3', 'ENOX2', 'F8', 'FAM120C', 'FUNDC2', 'H2AFB3', 
    'LANCL3', 'LAS1L', 'LOC114675151', 'LOC114675176', 'LOC114675180', 'LOC114675218', 
    'LOC114675231', 'LOC114675302', 'LOC695959', 'LOC696657', 'LOC703257', 'LOC706958', 
    'MAGT1', 'MIR188', 'MIR362', 'MIR500A', 'MIR500B', 'MIR501', 'MIR502', 'MIR532', 'MIR660',
    'MIR7206', 'MPP1', 'MSN', 'MTCP1', 'PAGE4', 'RAP2C', 'SH3KBP1', 'SMIM9', 'TRPC5', 'USP27X', 
    'WNK3', 'XK', 'ZC3H12B' 
])
sojern_genes

In [ ]:
test_lists = [
 # 'xi_escape',
 # 'meritxell_spermatid_expr',
    
 # 'expr_mod_xi_copynr_fibrobl',
     # 'accel_reg_simiiformes_br',
 # 'my_primate_codeml',

# 'ari_nonPUR',
    'cDEG',
    'nDEG',
    'xi_any_evidence',
 #     'mult_copy',
    

]


gene_list = []
for list_name in test_lists:
    gene_list.extend(sheet.get(list_name))
df = pd.DataFrame(dict(genes=sorted(set(gene_list))))
for list_name in test_lists:
    df[list_name] = df.genes.isin(sheet.get(list_name))

# df['sojern_genes'] = df.genes.isin(sojern_genes)
df['hama_genes'] = df.genes.isin(hama_genes)

# df = df.set_index(test_lists + ['sojern_genes', 'hama_genes'])
df = df.set_index(test_lists + ['hama_genes'])

In [ ]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# from upsetplot import UpSet
# ax_dict = UpSet(df.genes, show_counts=True, subset_size="count").plot()

In [ ]:
hama_genes.fisher(sheet.get('cDEG'), sheet.get('all_npx'))

## Try to load the matrices and look at them

In [ ]:
# import standard python libraries
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import os, subprocess

In [ ]:
# Import python package for working with cooler files and tools for analysis
import cooler
import cooltools.lib.plotting

# had to add this to _register_colormaps in /Users/kmt/miniconda3/envs/cooler/lib/python3.10/site-packages/cooltools/lib/plotting.py
# from matplotlib.colors import ListedColormap
# mpl.colormaps.register(ListedColormap(pal/255), name=name)
# mpl.colormaps.register(ListedColormap(pal[::-1]/255), name=name + "_r")


import cooltools
import bioframe
import pandas as pd
import os.path as op

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
clr_path = "../data/cooler/sperm.mcool"
res = 100000
clr = cooler.Cooler(f"{clr_path}::resolutions/{res}")
matrix = clr.matrix(balance=True).fetch('chrX') 

In [ ]:
# Make the full view frame
view_df = pd.DataFrame({
    'chrom': clr.chromnames[-2],
    # 'start': [40_000_000],
    # 'end': [100_000_000],
    # 'start': [40_000_000],
    # 'end': [55_000_000],
    'start': [0],
    'end': clr.chromsizes.values[-2],
    'name': clr.chromnames[-2]})

In [ ]:
bins = clr.bins().fetch('chrX')[:]
out_name = f'rheMac10_gc_cov_X_res{res//1000}kb.tsv'

rheMac10 = bioframe.load_fasta('../data/rheMac10.fa')
if not op.exists(out_name):
    print('Calculate the fraction of GC basepairs for each bin')
    gc_cov = bioframe.frac_gc(bins[['chrom', 'start', 'end']], rheMac10)
    gc_cov.to_csv(out_name, index=False,sep='\t')
    print(gc_cov.info())
else: 
    print("Already exists, read from file")
    gc_cov = pd.read_csv(out_name, sep='\t')
    print(gc_cov.info())

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

import umap

In [ ]:
import pywt
import matplotlib

mat = clr.matrix().fetch('chrX')
mat = mat[~np.isnan(mat)]
mat.shape

In [ ]:
#| label: fig-AB-macaque

import pywt
import matplotlib

pc = 1

# first 3 eigenvectors
cis_eigs = cooltools.eigs_cis(clr, gc_cov, view_df=view_df, n_eigs=3)

# cis_eigs[0] are eigenvalues
# cis_eigs[1] are eigenvectors
eigenvector_track = cis_eigs[1][['chrom','start','end',f'E{pc}']]
  
eigenvector_track_chrX = eigenvector_track.loc[eigenvector_track['chrom'] == 'chrX']
nbins = len(eigenvector_track_chrX)
eigenvector_track_chrX
eX_values = eigenvector_track_chrX[f'E{pc}'].values

f, ax = plt.subplots(figsize=(14, 3))
s = np.arange(len(eX_values))
x = np.dstack((s, s+1)).flatten()
y = np.repeat(eX_values, 2)
ax.fill_between(x, y, 0, where=(y > 0), ec='none', fc='tab:red')
ax.fill_between(x, y, 0, where=(y < 0), ec='none', fc='tab:blue')

In [ ]:
#| label: fig-AB-macaque-with-annot


import pywt
import matplotlib

pc = 1

# first 3 eigenvectors
cis_eigs = cooltools.eigs_cis(clr, gc_cov, view_df=view_df, n_eigs=3)

# cis_eigs[0] are eigenvalues
# cis_eigs[1] are eigenvectors
eigenvector_track = cis_eigs[1][['chrom','start','end',f'E{pc}']]
  
eigenvector_track_chrX = eigenvector_track.loc[eigenvector_track['chrom'] == 'chrX']
nbins = len(eigenvector_track_chrX)
eigenvector_track_chrX
eX_values = eigenvector_track_chrX[f'E{pc}'].values

f, ax = plt.subplots(figsize=(14, 3))
s = np.arange(len(eX_values))
x = np.dstack((s, s+1)).flatten()
y = np.repeat(eX_values, 2)
ax.fill_between(x, y, 0, where=(y > 0), ec='none', fc='tab:red')
ax.fill_between(x, y, 0, where=(y < 0), ec='none', fc='tab:blue')

# ax.fill_between(range(len(eX_values)), eX_values, 0, where=(eX_values > 0), ec='none', fc='tab:red')
# ax.fill_between(range(len(eX_values)), eX_values, 0, where=(eX_values < 0), ec='none', fc='tab:blue')

ax.hlines(np.repeat(-0.5, len(high_olive_rhemac10.start/res)), high_olive_rhemac10.start/res, high_olive_rhemac10.end/res, lw=8, colors='tab:green', capstyle='butt')
ax.hlines(np.repeat(-0.7, len(high_hama_rhemac10.start/res)), high_hama_rhemac10.start/res, high_hama_rhemac10.end/res, lw=8, colors='tab:blue', capstyle='butt')
ax.hlines(np.repeat(-0.9, len(ech90_human_Mmul_10.start/res)), ech90_human_Mmul_10.start/res, ech90_human_Mmul_10.end/res, lw=8, colors='tab:red', capstyle='butt')
ax.set_ylabel(f'E{pc}')
# ax.set_xticks([]);
# ax.set_yticks([]);
ax.get_subplotspec()
# ax.set_axis_off()


# coords = gi.gene_labels(nDEG_genes, assembly='rheMac10')
# # coords = [(tup[1]+tup[2])/2 for name, tup in gi.gene_coord(nDEG_genes, species='macaca_mulatta').items()]
# for chrom, pos, name in coords:
#     ax.vlines(pos/100000,  -1, 1, lw=0.2, color='black')
coords = gi.gene_labels(sheet.get('cDEG'), assembly='rheMac10')
# coords = [(tup[1]+tup[2])/2 for name, tup in gi.gene_coord(cDEG_genes, assembly='rheMac10').items()]
for chrom, pos, name in coords:
    ax.vlines(pos/100000,  -1, 1, lw=0.4, color='magenta')



plt.tight_layout()

sns.despine()

In [ ]:

import pywt
import matplotlib
from matplotlib.axes import Axes

pc = 1

# first 3 eigenvectors
cis_eigs = cooltools.eigs_cis(clr, gc_cov, view_df=view_df, n_eigs=3)

# cis_eigs[0] are eigenvalues
# cis_eigs[1] are eigenvectors
eigenvector_track = cis_eigs[1][['chrom','start','end',f'E{pc}']]
  
eigenvector_track_chrX = eigenvector_track.loc[eigenvector_track['chrom'] == 'chrX']
nbins = len(eigenvector_track_chrX)
eigenvector_track_chrX
eX_values = eigenvector_track_chrX[f'E{pc}'].values


s = np.arange(len(eX_values))
x = np.dstack((s, s+1)).flatten() * 100_000
y = np.repeat(eX_values, 2)
plot_df = pd.DataFrame(dict(x=x, y=y))
plot_df['chrom'] = 'chrX'

In [ ]:
# import requests, re
# from collections import defaultdict

# def chrom_sort_key(chrom):
#     if isinstance(chrom, (list, tuple)):
#         chrom = chrom[0]
#     return [int(x) if x.isdigit() else x for x in re.split('(\d+)', chrom)]


# # assembly = 'hg38'
# # api_url = f'https://api.genome.ucsc.edu/getData/track?genome={assembly};track=centromeres'
# # response = requests.get(api_url)
# # if not response.ok:
# #     response.raise_for_status()

# # data = defaultdict(list)
# # for chrom, val in response.json()['centromeres'].items():
# #     for d in val:
# #         data[d['chrom']].append((d['chromStart'], d['chromEnd']))
# # records = []
# # for ch, val in data.items():
# #     starts, ends = zip(*val)
# #     records.append((ch, min(starts), max(ends)))
# # df = pd.DataFrame().from_records(records, columns=['chrom', 'start', 'end'])
# # df.sort_values('chrom', key=lambda sr: [chrom_sort_key(x) for x in sr]).reset_index(drop=True)

# def centromeres(assembly):
#     api_url = f'https://api.genome.ucsc.edu/getData/track?genome={assembly};track=centromeres'
#     response = requests.get(api_url)
#     if not response.ok:
#         return response.raise_for_status()

#     data = defaultdict(list)
#     for chrom, val in response.json()['centromeres'].items():
#         for d in val:
#             data[d['chrom']].append((d['chromStart'], d['chromEnd']))
#     records = []
#     for ch, val in data.items():
#         starts, ends = zip(*val)
#         records.append((ch, min(starts), max(ends)))
#     return sorted(records, key=chrom_sort_key)  

# centromeres('hg38')

In [ ]:
#| label: fig-gog-pure-reions-cDEG


g = gplt.ChromIdeogram('chrX', assembly='papAnu4', 
                    font_size=3,
                       zoom_height_ratio=2, 
                       zoom_font_size=5, 
                       zoom_effect_alpha=0.2,
#                       ylim=(0, 10),
                          zooms=[
        (10_000_000, 20_000_000),
        (33_000_000, 54_000_000),
        (60_000_000, 63_000_000),
        (85_000_000, 120_000_000),
        (140_000_000, 145_000_000),
    ]
                      )

g.draw_chromosomes(base=2)
g.add_segments([(t.chrom, t.start, t.end) for t in olive_pure_segments.itertuples()], 
               facecolor='tab:green', base=3, height=1, label='Pure olive', alpha=0.7)
g.add_segments([(t.chrom, t.start, t.end) for t in hama_pure_segments.itertuples()], 
               facecolor='tab:orange', base=2, height=1, label='Pure hamadryas', alpha=0.7)


# ax.fill_between(x, y, 0, where=(y > 0), ec='none', fc='tab:red')
# ax.fill_between(x, y, 0, where=(y < 0), ec='none', fc='tab:blue')

# def color(name):
#     return 'black' if name in hum_nean_genes else 'black'

#gi.gene_coords(['DYNLT3', 'CFAP47'], assembly='papAnu4')

coords = gi.gene_labels(sheet.get('cDEG'), assembly='papAnu4')

# coords = []
# coords.extend(
#     [(tup[0], (tup[1]+tup[2])/2, name) for name, tup in gi.gene_coord(cDEG_genes, species='papio_anubis', assembly='papAnu4').items()]
# )

# #plot_df['y1'] = 1
# g.map_method(Axes.fill_between, data=plot_df, x='x', y='y', y2=1, where=(y > 0), ec='none', fc='tab:red', yaxis=(1, 2), label='A compartment')
# g.map_method(Axes.fill_between, data=plot_df, x='x', y='y', y2=1, where=(y <= 0), ec='none', fc='tab:blue', yaxis=(1, 2), label='B compartment')

#g.map_method(Axes.plot, data=plot_df, x='x', y='y', yaxis=(1, 2), label='A compartment')

# ax.fill_between(x, y, 0, where=(y > 0), ec='none', fc='tab:red')
# ax.fill_between(x, y, 0, where=(y < 0), ec='none', fc='tab:blue')

# g.add_horizon(plot_df, cut=0.1, palette='bluered_r')

# new_ax = g.add_axes(1, hspace=0.5)
# new_ax.fill_between(x, y, 0, where=(y > 0), ec='none', fc='tab:red')
# new_ax.fill_between(x, y, 0, where=(y < 0), ec='none', fc='tab:blue')

#coords = [(tup[0], (tup[1]+tup[2])/2, name) for name, tup in gi.gene_coord(cDEG_genes, species='papio_anubis', assembly='papAnu4').items()]
#g.add_labels(coords)
g.add_labels(coords, base=g.ideogram_base, min_height=g.ideogram_height*1.5, 
             zoom_base=g.ideogram_base, zoom_min_height=g.ideogram_height*1.2, color='black')
g.add_legend()

In [ ]:
sheet.get('cDEG') << (sheet.get('ech90_regions') | sheet.get('hum_nean_admix')) << sheet.get('ari_relate_AFR') << sheet.get('accel_reg_simiiformes_br')

In [ ]:
#| label: fig-macaque-hic-matrix

import pywt
import matplotlib

pc = 1

# first 3 eigenvectors
cis_eigs = cooltools.eigs_cis(clr, gc_cov, view_df=view_df, n_eigs=3)

# cis_eigs[0] are eigenvalues
# cis_eigs[1] are eigenvectors
eigenvector_track = cis_eigs[1][['chrom','start','end',f'E{pc}']]
  
eigenvector_track_chrX = eigenvector_track.loc[eigenvector_track['chrom'] == 'chrX']
nbins = len(eigenvector_track_chrX)
eigenvector_track_chrX
eX_values = eigenvector_track_chrX[f'E{pc}'].values

f, ax = plt.subplots(figsize=(9, 9))
# norm = LogNorm(vmax=0.1)
norm = LogNorm(vmax=0.005)

mat = clr.matrix().fetch('chrX')
lower = np.tril(mat, -1)

mat = clr.matrix().fetch('chrX')
base = np.nanmin(mat[np.nonzero(mat)])
_mat = np.log(mat + base) / np.log(base)
mra2 = pywt.mra2(_mat, wavelet='haar')
_mat, decomp = mra2[0], mra2[1]
_mat = base**_mat
upper = np.triu(_mat, 1)

n = len(upper) + 1
ul_mat = np.eye(n)
ul_mat[np.triu_indices(n, 1)] = upper[np.triu_indices(n-1)]
ul_mat[np.tril_indices(n, -1)] = lower[np.tril_indices(n-1)]

masked_array = ul_mat
cmap = matplotlib.cm.YlOrRd  
cmap.set_bad('black', alpha=0.1)
im = ax.matshow(masked_array, norm=norm, 
               interpolation='nearest', 
               cmap=cmap)

plt.axis([0,nbins,nbins,0])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.2)
plt.colorbar(im, cax=cax, label='corrected frequencies');
ax.set_ylabel('Position in Mb')
yticks = np.array(ax.get_yticks())
yticks, ylabels = zip(*[(y, l) for (y, l) in zip(yticks, (yticks * res + view_df.start[0]) / 1e6) if y < mat.shape[0]])
ax.set_yticks(yticks, ylabels)
ax.xaxis.set_visible(False)
sns.despine()

ax1 = divider.append_axes("top", size="25%", pad=0.05, sharex=ax)
#weights = clr.bins()[:]['weight'].values
#ax1.plot([0,nbins],[0,0],'k',lw=0.25)

s = np.arange(len(eX_values))
x = np.dstack((s, s+1)).flatten()
y = np.repeat(eX_values, 2)
ax1.fill_between(x, y, 0, where=(y > 0), ec='none', fc='tab:red')
ax1.fill_between(x, y, 0, where=(y < 0), ec='none', fc='tab:blue')

# ax1.fill_between(range(len(eX_values)), eX_values, 0, where=(eX_values > 0), ec='none', fc='tab:red')
# ax1.fill_between(range(len(eX_values)), eX_values, 0, where=(eX_values < 0), ec='none', fc='tab:blue')

ax1.hlines(np.repeat(-0.5, len(high_olive_rhemac10.start/res)), high_olive_rhemac10.start/res, high_olive_rhemac10.end/res, lw=8, colors='tab:green', capstyle='butt')
ax1.hlines(np.repeat(-0.7, len(high_hama_rhemac10.start/res)), high_hama_rhemac10.start/res, high_hama_rhemac10.end/res, lw=8, colors='tab:blue', capstyle='butt')
ax1.hlines(np.repeat(-0.9, len(ech90_human_Mmul_10.start/res)), ech90_human_Mmul_10.start/res, ech90_human_Mmul_10.end/res, lw=8, colors='tab:red', capstyle='butt')
ax1.set_ylabel(f'E{pc}')
ax1.set_xticks([]);
ax1.set_yticks([]);
ax1.get_subplotspec()
ax1.set_axis_off()

# for s, e in zip(olive_pure_segments.start, olive_pure_segments.end):
#     ax.vlines(olive_pure_segments.start/100000, 0, nbins, lw=0.1, color='tab:green')
#     ax.vlines(olive_pure_segments.end/100000, 0, nbins, lw=0.1, cocDEG_geneslor='tab:green')
#     ax.hlines(olive_pure_segments.start/100000, 0, nbins, lw=0.1, color='tab:green')
#     ax.hlines(olive_pure_segments.end/100000, 0, nbins, lw=0.1, color='tab:green')
    
# for s, e in zip(high_hama_rhemac10.start, high_hama_rhemac10.end):
#     ax.vlines(high_hama_rhemac10.start/res, 0, nbins, lw=0.1, color='tab:blue')
#     ax.vlines(high_hama_rhemac10.end/res, 0, nbins, lw=0.1, color='tab:blue')
#     ax.hlines(high_hama_rhemac10.start/res, 0, nbins, lw=0.1, color='tab:blue')
#     ax.hlines(high_hama_rhemac10.end/res, 0, nbins, lw=0.1, color='tab:blue')

# for s, e in zip(ech90_human_Mmul_10.start, ech90_human_Mmul_10.end):
#     ax.vlines(ech90_human_Mmul_10.start/100000, 0, nbins, lw=0.1, color='tab:brown')
#     ax.vlines(ech90_human_Mmul_10.end/100000, 0, nbins, lw=0.1, color='tab:brown')
#     ax.hlines(ech90_human_Mmul_10.start/100000, 0, nbins, lw=0.1, color='tab:brown')
#     ax.hlines(ech90_human_Mmul_10.end/100000, 0, nbins, lw=0.1, color='tab:brown')

#     ax1.vlines(ech90_human_Mmul_10.start/100000,  -1, 1, lw=0.1, color='tab:brown')
#     ax1.vlines(ech90_human_Mmul_10.end/100000,  -1, 1, lw=0.1, color='tab:brown')


# coords = [(tup[1]+tup[2])/2 for name, tup in gi.gene_coord(nDEG_genes, species='macaca_mulatta').items()]
coords = gi.gene_labels(sheet.get('cDEG'), assembly='rheMac10')
for chrom, pos, name in coords:
    ax.vlines(pos/100000, 0, nbins, lw=0.5, color='black')
    ax.hlines(pos/100000, 0, nbins, lw=0.5, color='black')

    ax1.vlines(pos/100000,  -1, 1, lw=0.5, color='black')



plt.tight_layout()

# for i in np.where(np.diff( (cis_eigs[1][cis_eigs[1]['chrom']=='chrX'][f'E{pc}']>0).astype(int)))[0]:
#     ax.plot([0,nbins],[i,i],'k',lw=0.2)
#     ax.plot([i,i],[0,nbins],'k',lw=0.2)
    
#     ax1.plot([i,i],[0, -1],'k',lw=0.2)

In [ ]:
upper = np.triu(mat, 0)
lower = np.tril(mat, -1)

n = len(upper) + 1
out = np.eye(n)
out[np.triu_indices(n, 1)] = upper[np.triu_indices(n-1)]
out[np.tril_indices(n, -1)] = lower[np.tril_indices(n-1)]
mat.shape

In [ ]:
clr_path = "../data/cooler/sperm.mcool"
res = 100000
clr = cooler.Cooler(f"{clr_path}::resolutions/{res}")
matrix = clr.matrix(balance=True).fetch('chrX') 

In [ ]:
# Make the full view frame
view_df = pd.DataFrame({
    'chrom': clr.chromnames[-2],
    'start': [0],
    'end': clr.chromsizes.values[-2],
    'name': clr.chromnames[-2]})

In [ ]:
bins = clr.bins().fetch('chrX')[:]
out_name = f'../data/rheMac10_gc_cov_X_res{res//1000}kb.tsv'

rheMac10 = bioframe.load_fasta('../data/rheMac10.fa')
if not op.exists(out_name):
    print('Calculate the fraction of GC basepairs for each bin')
    gc_cov = bioframe.frac_gc(bins[['chrom', 'start', 'end']], rheMac10)
    gc_cov.to_csv(out_name, index=False,sep='\t')
    print(gc_cov.info())
else: 
    print("Already exists, read from file")
    gc_cov = pd.read_csv(out_name, sep='\t')
    print(gc_cov.info())

In [ ]:
pc = 1

# first 3 eigenvectors
cis_eigs = cooltools.eigs_cis(clr, gc_cov, view_df=view_df, n_eigs=3)

# cis_eigs[0] are eigenvalues
# cis_eigs[1] are eigenvectors
eigenvector_track = cis_eigs[1][['chrom','start','end',f'E{pc}']]

# subset the chrX   
eigenvector_track_chrX = eigenvector_track.loc[eigenvector_track['chrom'] == 'chrX']
nbins = len(eigenvector_track_chrX)
eigenvector_track_chrX
eX_values = eigenvector_track_chrX[f'E{pc}'].values

f, ax = plt.subplots(figsize=(5, 5))
norm = LogNorm(vmax=0.1)
im = ax.matshow(clr.matrix().fetch('chrX'), norm=norm, cmap='viridis')

In [ ]:
mat = clr.matrix().fetch('chrX')
mat = np.log(mat + 1e-5) / np.log(1e-5)
fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(mat)

In [ ]:
pywt.wavelist('haar')

In [ ]:
mra2 = pywt.mra2(mat, wavelet='haar')
approx, decomp = mra2[0], mra2[1]
fig, axes = plt.subplots(1, len(mra2), figsize=(12, 5))
axes[0].matshow(approx, cmap='viridis')
for ar, ax in zip(decomp, axes[1:]):
    ax.matshow(ar, cmap='fall')

In [ ]:
# Import python package for working with cooler files and tools for analysis
import cooler
import cooltools.lib.plotting
import cooltools

mclr = "../data/cooler/sperm.mcool"

clr = cooler.Cooler(f"{mclr}::resolutions/100000")

In [ ]:
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

f, ax = plt.subplots(
    figsize=(6, 6),
)

norm = LogNorm(vmax=0.1)

im = ax.matshow(
    clr.matrix().fetch('chrX'),
    norm=norm,
    cmap='fall',
);
#plt.axis([0,nbins,nbins,0])


divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax, label='corrected frequencies');
ax.set_ylabel('chrX:500kb bins, #bin')
ax.xaxis.set_visible(False)

# ax1 = divider.append_axes("top", size="20%", pad=0.05, sharex=ax)
# #weights = clr.bins()[:]['weight'].values
# #ax1.plot([0,nbins],[0,0],'k',lw=0.25)

# #ax1.plot(e1X_values, label='E1')

# # Fill between the line and 0
# ax1.fill_between(range(len(e1X_values)), e1X_values, 0, where=(e1X_values > 0), color='tab:red')
# ax1.fill_between(range(len(e1X_values)), e1X_values, 0, where=(e1X_values < 0), color='tab:blue')

# ax1.set_ylabel('E1')
# ax1.set_xticks([]);
# ax1.get_subplotspec()


# for i in np.where(np.diff( (cis_eigs[1][cis_eigs[1]['chrom']=='chrX']['E1']>0).astype(int)))[0]:
#     # Horisontal lines where E1 intersects 0
#     ax.plot([0,nbins],[i,i],'k',lw=0.4)

#     # Vertical lines where E1 intersects 0
#     ax.plot([i,i],[0,nbins],'k',lw=0.4)

## UMAP

In [ ]:
# penguins = pd.read_csv("https://raw.githubusercontent.com/allisonhorst/palmerpenguins/c19a904462482430170bfe2c718775ddb7dbb885/inst/extdata/penguins.csv")
# penguins.head()

In [ ]:
np.isnan(mat)

In [ ]:
#penguins = penguins.dropna()
mat = clr.matrix().fetch('chrX')

# mask = ~np.isnan(mat).any(axis=1)
mat = mat[~np.isnan(mat).all(axis=1), :]
# mat = mat[:, ~np.isnan(mat).all(axis=0)]
mat = mat[:, ~np.isnan(mat).all(axis=0)]
mat

In [ ]:
reducer = umap.UMAP()

In [ ]:
# penguin_data = penguins[
#     [
#         "bill_length_mm",
#         "bill_depth_mm",
#         "flipper_length_mm",
#         "body_mass_g",
#     ]
# ].values
# penguin_data

In [ ]:
#scaled_penguin_data = StandardScaler().fit_transform(penguin_data)
# scaled_penguin_data
scaled_matrix = StandardScaler().fit_transform(mat)
scaled_matrix

In [ ]:
# embedding = reducer.fit_transform(scaled_penguin_data)
embedding = reducer.fit_transform(scaled_matrix)
embedding.shape

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=np.arange(0, embedding.shape[0]),
    cmap='rainbow',
    s=5
    # c=[sns.color_palette()[x] for x in penguins.species.map({"Adelie":0, "Chinstrap":1, "Gentoo":2})]
)
plt.gca().set_aspect('equal', 'datalim')

In [ ]:
mapper = umap.UMAP().fit(scaled_matrix)

umap.plot.points(mapper);

In [ ]:
import umap.plot
umap.plot.output_notebook()

In [ ]:
umap.plot.connectivity(mapper, show_points=True);

In [ ]:
umap.plot.connectivity(mapper, show_points=True, edge_bundling='hammer')

In [ ]:
np.arange(0,scaled_matrix.shape[0]

mat

In [ ]:
umap.plot.points(mapper, values=np.arange(0, scaled_matrix.shape[0]), cmap='rainbow')
umap.plot.connectivity(mapper, show_points=True, edge_bundling='hammer')

In [ ]:
umap.plot.diagnostic(mapper, diagnostic_type='pca') ; 


In [ ]:
umap.plot.diagnostic(mapper, diagnostic_type='vq') ;

In [ ]:
local_dims = umap.plot.diagnostic(mapper, diagnostic_type='local_dim') ;

In [ ]:
umap.plot.diagnostic(mapper, diagnostic_type='neighborhood') ;

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=np.arange(0, embedding.shape[0]),
    cmap='rainbow'
    # c=[sns.color_palette()[x] for x in penguins.species.map({"Adelie":0, "Chinstrap":1, "Gentoo":2})]
)
plt.gca().set_aspect('equal', 'datalim')



## Try to make plot recombination rate to see if it is lower in pure regions